# RLHF From Scratch on LLMs

In this notebook, I will start with history of RLHF, the importance of RLHF in LLMs, then go into the architectures TRPO, PPO, GRPO and DPO. Each of the technique's explanation will have the math, code and explanations on how it's done, finally in the end we'll experiment these techniques on one of prebuilt LLMs (the llms are not built from scratch, since i've already done that in my [llm-from-scratch repository](https://github.com/ashworks1706/llm-from-scratch)) 

### Brief History of RLHF

RLHF emerged around 2017-2018 when researchers at OpenAI developed techniques to incorporate human preferences into reinforcement learning systems. The seminal paper "Deep Reinforcement Learning from Human Preferences" by Christiano et al. (2017) introduced the core concept of using human comparisons between pairs of outputs to train a reward model that could guide RL agents toward preferred behaviors. While initially applied to simpler tasks and robotics, the technique remained relatively specialized until recent years. The technique gained mainstream attention in 2022 when OpenAI used it to create ChatGPT from GPT-3.5, dramatically improving output quality by aligning the model with human preferences. This breakthrough demonstrated RLHF's potential to transform raw language model capabilities into systems that better align with human intent and values. Since then, RLHF has become a standard component in developing advanced language models like GPT-4, Claude, and Llama 2, with each iteration refining the techniques to achieve better alignment.

#### Why RLHF Matters for LLMs

<img src="assets/rlhf-vs-finetune.png" width=300>

Large Language Models trained solely on next-token prediction are just models with knowledge, they don't know how to answer properly. You have model trained on shakespear work, great! but how do you make it to answer questions in the way we want (in the way humans talk)? These models optimize for predicting the next token based on training data distribution, which doesn't necessarily correlate with producing helpful, harmless, or honest responses. Traditional LLMs may generate toxic, harmful, or misleading content because they're simply trying to produce statistically likely continuations without understanding human values or preferences. They lack an inherent mechanism to distinguish between content that is statistically probable and content that is actually desirable according to human standards. RLHF addresses these issues by creating a feedback loop where human preferences explicitly guide the model's learning process, steering it toward outputs that humans find more helpful, honest, and aligned with their intent. This alignment process transforms a powerful but directionless prediction engine into a system that can better understand and respect nuanced human values and follow complex instructions in ways that maximize utility for users.




### Workflow - The Birds Eye View

<img src="assets/workflow.png" >


Before delving into the complexities of RLHF, it's essential to understand the crucial role that Supervised Fine-Tuning (SFT) plays in transforming a raw language model into something more aligned with human expectations. When a large language model is initially trained on vast internet text corpora, it develops remarkable capabilities to predict text and acquire factual knowledge, but this training alone doesn't prepare it to be helpful in specific contexts or respond appropriately to human instructions. Consider a language model trained on extensive educational materials, including university canvas modules, academic papers, and textbooks. This model would possess substantial knowledge about various academic subjects, pedagogical approaches, and educational concepts. However, if asked to "Explain the concept of photosynthesis to a 10-year-old," it might produce a technically accurate but overly complex explanation filled with academic jargon that would confuse rather than enlighten a young student. The model hasn't been optimized to serve as an effective tutor - it simply predicts what text might follow in educational materials. 

The Supervised Fine-Tuning stage addresses this gap by training the model on demonstrations of desired behavior. For our hypothetical educational assistant, SFT would involve collecting thousands of examples showing how skilled human tutors respond to student questions: simplifying complex concepts, using age-appropriate language, providing relevant examples, checking for understanding, and offering encouragement. These demonstrations are formatted as input-output pairs (prompt and ideal response), and the model is fine-tuned to minimize the difference between its outputs and these human-generated "gold standard" responses. Through this process, the model learns the patterns that characterize helpful tutoring: breaking down complex concepts into simpler components, using analogies relevant to younger audiences, avoiding unnecessary technical terms, and adopting a supportive tone. After SFT, when asked to explain photosynthesis to a 10-year-old, the model is much more likely to respond with an explanation involving plants "eating sunlight" and "breathing in carbon dioxide to make food," rather than discussing electron transport chains and ATP synthesis. The model hasn't gained new knowledge, but it has learned a new way to present its existing knowledge that better aligns with the specific goal of being an effective tutor for younger students. However, SFT alone has significant limitations. First, it can only learn from the specific examples it's shown, leaving gaps in how to handle the infinite variety of possible user requests. Second, the demonstrations might not cover the full range of desirable behaviors or edge cases where special handling is needed. Third, the quality of the SFT model depends entirely on the quality and consistency of the demonstration data. Finally, there's no mechanism for the model to understand why certain responses are better than others - it simply learns to mimic patterns without a deeper understanding of the preferences that make one response superior. These limitations are precisely what RLHF is designed to address in the subsequent stages of the alignment process.
